In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split

from tensorflow import keras
from keras.layers import LSTM, Bidirectional
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.layers import Flatten
from keras.layers.convolutional import MaxPooling1D

from nltk.corpus import stopwords
from nltk import word_tokenize



In [ ]:
depression_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/roman_urdu_all_data.csv')
depression_data = depression_data.astype(str)

In [ ]:
depression_data.shape

(9526, 8)

In [ ]:
x = np.array(depression_data['text'])
y = np.array(depression_data['Label'])
y = y.astype(float)
y = y.astype(int)

In [ ]:
c=0
max_features = 50000
maxlen = 150
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 10
P=[]
R=[]
F=[]

In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 50
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(depression_data['text'].values)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [ ]:
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [ ]:
print(np.unique(y_test))

[0 1]


# CNN

In [ ]:
import tensorflow as tf 
model = Sequential() 
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
# model.add(Conv1D(32, 3, padding='same', activation='relu'))
# model.add(MaxPooling1D())
# model.add(Flatten())
# model.add(Dense(250, activation='relu'))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.summary()

model.add(Conv1D(filters=128, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 150, 50)           2500000   
                                                                 
 conv1d (Conv1D)             (None, 150, 128)          25728     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 75, 128)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 75, 64)            32832     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 37, 64)           0         
 1D)                                                             
                                                                 
 conv1d_2 (Conv1D)           (None, 37, 32)            8

In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)
# Getting score metrics from our model
scores = model.evaluate(X_test, y_test, verbose=0)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
200/200 - 18s - loss: 0.3861 - accuracy: 0.8330 - val_loss: 0.2562 - val_accuracy: 0.8375 - 18s/epoch - 89ms/step
Epoch 2/10
200/200 - 16s - loss: 0.1673 - accuracy: 0.9243 - val_loss: 0.2498 - val_accuracy: 0.8820 - 16s/epoch - 80ms/step
Epoch 3/10
200/200 - 17s - loss: 0.0779 - accuracy: 0.9712 - val_loss: 0.3015 - val_accuracy: 0.8849 - 17s/epoch - 85ms/step
Epoch 4/10
200/200 - 18s - loss: 0.0436 - accuracy: 0.9867 - val_loss: 0.3206 - val_accuracy: 0.8795 - 18s/epoch - 92ms/step
Epoch 5/10
200/200 - 16s - loss: 0.0286 - accuracy: 0.9914 - val_loss: 0.4289 - val_accuracy: 0.8775 - 16s/epoch - 79ms/step
Epoch 6/10
200/200 - 16s - loss: 0.0162 - accuracy: 0.9956 - val_loss: 0.4651 - val_accuracy: 0.8795 - 16s/epoch - 79ms/step
Epoch 7/10
200/200 - 16s - loss: 0.0117 - accuracy: 0.9967 - val_loss: 0.5390 - val_accuracy: 0.8766 - 16s/epoch - 80ms/step
Epoch 8/10
200/200 - 16s - loss: 0.0121 - accuracy: 0.9969 - val_loss: 0.5131 - val_accuracy: 0.8725 - 16s/epoch - 79ms/step


In [ ]:
CNN_test_pridict = model.predict(X_test) 
arr = []
for i in range(len(CNN_test_pridict)):
  if CNN_test_pridict[i]>=0.5:
    arr.append(1)
  else:
    arr.append(0)
print(len(arr))
print(len(y_test))

3144
3144


In [ ]:
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test,arr)
print ("fbeta_score on Testing Data is:", fbeta_score)

fbeta_score on Testing Data is: [0.91765622 0.63249348]


# LSTM

In [ ]:
model = Sequential() 
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 150, 50)           2500000   
                                                                 
 lstm_2 (LSTM)               (None, 128)               91648     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_2 (Dense)             (None, 32)                4128      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2,595,809
Trainable params: 2,595,809
No

In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)
# Getting score metrics from our model
scores = model.evaluate(X_test, y_test, verbose=0)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
200/200 - 93s - loss: 0.3907 - accuracy: 0.8372 - val_loss: 0.2198 - val_accuracy: 0.8842 - 93s/epoch - 467ms/step
Epoch 2/10
200/200 - 92s - loss: 0.1728 - accuracy: 0.9246 - val_loss: 0.2348 - val_accuracy: 0.8852 - 92s/epoch - 460ms/step
Epoch 3/10
200/200 - 93s - loss: 0.1149 - accuracy: 0.9575 - val_loss: 0.3124 - val_accuracy: 0.8877 - 93s/epoch - 463ms/step
Epoch 4/10
200/200 - 89s - loss: 0.0765 - accuracy: 0.9723 - val_loss: 0.3385 - val_accuracy: 0.8779 - 89s/epoch - 444ms/step
Epoch 5/10
200/200 - 90s - loss: 0.0521 - accuracy: 0.9842 - val_loss: 0.3999 - val_accuracy: 0.8836 - 90s/epoch - 449ms/step
Epoch 6/10
200/200 - 91s - loss: 0.0396 - accuracy: 0.9879 - val_loss: 0.4482 - val_accuracy: 0.8820 - 91s/epoch - 456ms/step
Epoch 7/10
200/200 - 89s - loss: 0.0298 - accuracy: 0.9903 - val_loss: 0.4641 - val_accuracy: 0.8747 - 89s/epoch - 444ms/step
Epoch 8/10
200/200 - 89s - loss: 0.0195 - accuracy: 0.9931 - val_loss: 0.5315 - val_accuracy: 0.8693 - 89s/epoch - 443

In [ ]:
CNN_test_pridict = model.predict(X_test) 
arr = []
for i in range(len(CNN_test_pridict)):
  if CNN_test_pridict[i]>=0.5:
    arr.append(1)
  else:
    arr.append(0)
print(len(arr))
print(len(y_test))

3144
3144


In [ ]:
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test,arr)
print ("fbeta_score on Testing Data is:", fbeta_score)

fbeta_score on Testing Data is: [0.9161087  0.64793388]


# Bi-LSTM

In [ ]:
model = Sequential() 
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
model.add(Bidirectional(LSTM(128, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',  metrics=[tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 150, 50)           2500000   
                                                                 
 bidirectional (Bidirectiona  (None, 256)              183296    
 l)                                                              
                                                                 
 dense_7 (Dense)             (None, 1)                 257       
                                                                 
Total params: 2,683,553
Trainable params: 2,683,553
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test), verbose=2)
# Getting score metrics from our model
scores = model.evaluate(X_test, y_test, verbose=0)
# Displays the accuracy of correct sentiment prediction over test data
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
200/200 - 163s - loss: 0.3582 - precision_3: 0.6270 - recall_3: 0.3520 - val_loss: 0.2163 - val_precision_3: 0.6929 - val_recall_3: 0.6424 - 163s/epoch - 817ms/step
Epoch 2/10
200/200 - 154s - loss: 0.1731 - precision_3: 0.7645 - recall_3: 0.8590 - val_loss: 0.2125 - val_precision_3: 0.6894 - val_recall_3: 0.7205 - 154s/epoch - 771ms/step
Epoch 3/10
200/200 - 155s - loss: 0.1232 - precision_3: 0.8471 - recall_3: 0.9044 - val_loss: 0.2392 - val_precision_3: 0.7094 - val_recall_3: 0.6823 - 155s/epoch - 774ms/step
Epoch 4/10
200/200 - 154s - loss: 0.0791 - precision_3: 0.9155 - recall_3: 0.9373 - val_loss: 0.2845 - val_precision_3: 0.6878 - val_recall_3: 0.7153 - 154s/epoch - 770ms/step
Epoch 5/10
200/200 - 154s - loss: 0.0558 - precision_3: 0.9398 - recall_3: 0.9530 - val_loss: 0.3328 - val_precision_3: 0.7165 - val_recall_3: 0.5573 - 154s/epoch - 769ms/step
Epoch 6/10
200/200 - 156s - loss: 0.0364 - precision_3: 0.9633 - recall_3: 0.9744 - val_loss: 0.4062 - val_precision_3: 

In [ ]:
CNN_test_pridict = model.predict(X_test) 
arr = []
for i in range(len(CNN_test_pridict)):
  if CNN_test_pridict[i]>=0.5:
    arr.append(1)
  else:
    arr.append(0)
print(len(arr))
print(len(y_test))

3144
3144


In [ ]:
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test,arr)
print ("fbeta_score on Testing Data is:", fbeta_score)

fbeta_score on Testing Data is: [0.92286926 0.63702359]
